In [46]:
import numpy as np
import time
import tritonclient.http as tritonhttpclient
import tritonclient.grpc as tritongrpcclient
from tqdm import tqdm
import argparse
import gevent.ssl
import tritonclient.grpc as grpcclient
import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException
import easydict
import argparse
import numpy as np
import sys
from builtins import range
from ctypes import *
import tritonclient.grpc as grpcclient
from tritonclient import utils
import tritonclient.utils.shared_memory as shm
import tritonclient.http as tritonhttpclient
import subprocess
import os
import requests
import re
from appdirs import user_cache_dir
from numpy import load
from appdirs import user_cache_dir
from numpy import asarray
from numpy import save
from tritonclient.utils import triton_to_np_dtype
!apt-get install vmtouch
!pip install appdirs


Reading package lists... Done
Building dependency tree       
Reading state information... Done
vmtouch is already the newest version (1.3.1-1).
0 upgraded, 0 newly installed, 0 to remove and 90 not upgraded.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [60]:
class IGirder:
    first = 0
    second = 0
    answer = 0

    # parameterized constructor
    def __init__(self, model_name, output_saved_model_dir, grpc_url, verbose, 
                 model_connection_url, server_health, configuration, batch_size,
                 input_name,output_name,model_version,url,
                 data_path, batch):
        self.model_name = model_name
        self.grpc_url = grpc_url
        self.verbose = verbose
        self.model_connection_url=model_connection_url
        self.server_health = server_health
        self.configuration=configuration
        self.output_saved_model_dir= output_saved_model_dir
        self.batch_size=batch_size
        self.input_name = input_name
        self.output_name = output_name
        self.output_name_1='output_0'
        self.model_version = model_version
        self.url = url
        self.data_path=data_path
        self.batch_copy= batch
        self.boolean = 'false'

    def protocol(self):
        self.triton_grpc_client = tritongrpcclient.InferenceServerClient(url=self.grpc_url, verbose=self.verbose)
        self.answer = self.first + self.second
    def model(self):
        output_saved_model_dir= self.output_saved_model_dir
    def model_connection_check(self):
        import requests
        res = requests.get('http://'+ str(self.model_connection_url))
        if (res.status_code == 200):
          print("Model Connection esteblished")
        else:
          print("Model Connection not esteblished")      

    def health(self):
        import requests
        server_health= subprocess.check_output(['curl', '-v',self.server_health])
        res = requests.get('http://' + str(self.server_health))
        if (res.status_code == 200):
            print("Server health esteblished")
        else:
            print("Server health not esteblished")    
    
    def instantiate_triton_client(self):
        self.triton_grpc_client = tritongrpcclient.InferenceServerClient(url=self.grpc_url, verbose=self.verbose)
        model_metadata = self.triton_grpc_client.get_model_metadata(model_name=self.model_name, model_version=self.model_version)
        model_config = self.triton_grpc_client.get_model_config(model_name=self.model_name, model_version=self.model_version)

    def batch(self):  
        from numpy import load
        # load array
        self.batch = load(self.data_path)

    def infer(self):
        from numpy import asarray
        from numpy import save
        import numpy as np
        # data loaded without using cache memory
        self.input1 = tritongrpcclient.InferInput(input_name, self.batch_copy.shape, 'FP16')
        self.input1.set_data_from_numpy(self.batch_copy)
        self.output1 = tritongrpcclient.InferRequestedOutput(self.output_name_1)

        # Infer from cache 
        self.input0 = tritongrpcclient.InferInput(input_name, self.batch.shape, 'FP16')

        self.input0.set_data_from_numpy(load(self.data_path))
        self.output = tritongrpcclient.InferRequestedOutput(self.output_name)
        
    def throughput(self):
        from functools import partial
        import sys

        if sys.version_info >= (3, 0):
            import queue
        else:
            import Queue as queue
        class UserData:
            def __init__(self):
                self._completed_requests = queue.Queue()
        def __init__(self):
            self._completed_requests = queue.Queue()
        def completion_callback(user_data, result, error):
            # passing error raise and handling out
            user_data._completed_requests.put((result, error))
        user_data = UserData()
        
        models = [model_name]

        async_requests = []
        responses = []
        request_count = 1000

        for i in tqdm(range(50)):
            async_requests.append(self.triton_grpc_client.async_infer(model_name, inputs=[self.input1],
                                                     callback=partial(completion_callback, user_data),
                                                               outputs=[self.output1]))

        print("Throughput with cache memory")
        start_time = time.time()

        for i in tqdm(range(request_count)):
                async_requests.append(self.triton_grpc_client.async_infer(model_name, inputs=[self.input0],
                                                     callback=partial(completion_callback, user_data),
                                                         outputs=[self.output]))
        sent_count = 1
        processed_count = 0
        while processed_count < sent_count:
            (results, error) = user_data._completed_requests.get()
            processed_count += 1
            if error is not None:
                print("inference failed: " + str(error))
                sys.exit(1)
            responses.append(user_data) 

        end_time = time.time()


        print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
        print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

In [62]:
tfr_files_path = '/tf/notebooks/Course-ece495/train'
npy_files_path = user_cache_dir("")
batch_size=2048
output_saved_model_dir = 'models/simple-trt-model-FP16/1/model.savedmodel'
model_connection_url = 'localhost:8000/v2/models/simple-trt-model-FP16/config'
server_health = 'localhost:8000/v2/health/ready'
input_name = 'input_0'
input_shape = (batch_size,1024)
input_dtype = 'FP16'
output_name = 'output_0'
model_name = 'simple-trt-model-FP16'
url = 'localhost:8000'
model_version = '1'

configuration = """
name: "simple-trt-model-FP16"
platform: "tensorflow_savedmodel"
max_batch_size: 2048

input [
 {
    name: "input_0"
    data_type: TYPE_FP16
    dims: [ 1024 ]
  }
]
output {
    name: "output_0"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }
instance_group [
    {
      count: 16
      kind: KIND_GPU
      gpus: [ 0,1]
    },
    {
      count: 16
      kind: KIND_GPU
      gpus: [2,3]
    },
    {
      count: 4
      kind: KIND_CPU
    }
  ]
dynamic_batching {
    
    max_queue_delay_microseconds: 200
  }
"""
with open('models/simple-trt-model-FP16/config.pbtxt', 'w') as file:
    file.write(configuration)
    
# Put the files in cache memory for fast access
def files_cache(s, pat=re.compile('100%')):
    if pat.search(s):
        print("Files already in Cache Memory")
    else:
        print("Adding Files in Cache Memory")
        subprocess.check_output(['vmtouch','-t',npy_files_path])        

#create numpy data
class iterator(object):

    def __init__(self, B=batch_size, D=1024):
        self.B = B # batch size
        self.D = D # dimension

    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        output = np.float16(np.random.uniform(size=(self.B, self.D)))

        return output
    
    def batch(self):     
        
        self.batch = (iterator(self.B))
        i = iter(self.batch)
        self.batch=next(i)
        return self.batch

# Generate a numpy array as npy file
obj = iterator()
batch =  obj.batch()
save(npy_files_path + '/batch.npy', batch)

# Adding numpy batch to cache memory
# for .tfr files change to npy_files_path = tfr_files_path
data_path = npy_files_path + '/batch.npy' # change to tfr_files_path for .tfr files
result= subprocess.check_output(['vmtouch',npy_files_path])
files_cache(str(result))

# creating object of the Girdir class
# this will invoke parameterized constructor
obj = IGirder(model_name,output_saved_model_dir,
             'localhost:8001',False,
             model_connection_url,server_health,
            configuration, batch_size,input_name,output_name,
            model_version,url,
            data_path,batch)
 
# perform Addition
obj.instantiate_triton_client()
# perform model connection
obj.model_connection_check()
#
obj.health()
obj.batch()
obj.infer()

Files already in Cache Memory
Model Connection esteblished
Server health esteblished


In [63]:
obj.throughput()

100%|██████████| 50/50 [00:00<00:00, 192.36it/s]


Throughput with cache memory


100%|██████████| 1000/1000 [00:04<00:00, 231.33it/s]

Average Latency: ~0.004327610015869141 seconds
Average Throughput: ~473240.42427346297 examples / second
